In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from datetime import date

In [2]:
data = yf.Ticker("^GSPC")
print(data.info)

{'exchange': 'SNP', 'shortName': 'S&P 500', 'exchangeTimezoneName': 'America/New_York', 'exchangeTimezoneShortName': 'EST', 'isEsgPopulated': False, 'gmtOffSetMilliseconds': '-18000000', 'quoteType': 'INDEX', 'symbol': '^GSPC', 'messageBoardId': 'finmb_INDEXGSPC', 'market': 'us_market', 'previousClose': 4477.44, 'regularMarketOpen': 4482.79, 'twoHundredDayAverage': 4444.2046, 'trailingAnnualDividendYield': None, 'payoutRatio': None, 'volume24Hr': None, 'regularMarketDayHigh': 4539.66, 'navPrice': None, 'averageDailyVolume10Day': 4006469000, 'totalAssets': None, 'regularMarketPreviousClose': 4477.44, 'fiftyDayAverage': 4619.071, 'trailingAnnualDividendRate': None, 'open': 4482.79, 'toCurrency': None, 'averageVolume10days': 4006469000, 'expireDate': None, 'yield': None, 'algorithm': None, 'dividendRate': None, 'exDividendDate': None, 'beta': None, 'circulatingSupply': None, 'startDate': None, 'regularMarketDayLow': 4451.5, 'priceHint': 2, 'currency': 'USD', 'regularMarketVolume': 2640944

In [3]:
# get historical market data
pd_underlying_price = data.history(period="10y")

pd_vol_surface = pd.read_csv("volatility_surface.csv")

pd_vol_surface['date'] = pd.to_datetime(pd_vol_surface['date'], format='%Y%m%d')

pd_vol_surface = pd_vol_surface.merge(pd_underlying_price[['Close']], left_on = 'date', right_index= True, how = 'left')

pd_vol_surface['moneyness'] = (pd_vol_surface['impl_strike']/pd_vol_surface['Close']-1)*100

In [4]:
year  = 2020
month = 3
day   = 17
put_call_flag = 'C'


In [5]:
pd_vol_surface_date = pd_vol_surface.loc[pd_vol_surface['date'].dt.date == date(year, month, day)]
pd_vol_surface_date = pd_vol_surface_date.loc[(pd_vol_surface_date['days'] > 10) & (pd_vol_surface_date['days'] <= 122) & (pd_vol_surface_date['cp_flag'] == put_call_flag)]

pd_vol_surface_date_group = pd_vol_surface_date.set_index(['days', 'moneyness'])
pd_vol_surface_date_group.sort_index(level=1, ascending=False, sort_remaining=False, inplace = True)
pd_vol_surface_date_group.sort_index(level=0, ascending=False, sort_remaining=False, inplace = True)
pd_vol_surface_date = pd_vol_surface_date_group.reset_index(level=[0,1])

N_T = len(pd_vol_surface_date.days.unique())

X = np.reshape(pd_vol_surface_date['days'].values,(N_T,-1)).T
Y = np.reshape(pd_vol_surface_date['moneyness'].values,(N_T,-1)).T
Z = np.reshape(pd_vol_surface_date['impl_volatility'].values,(N_T,-1)).T


In [36]:
from ipywidgets import interactive

def plt_vol_surface(E,A):

    fig = plt.figure(figsize = (12,8))
    ax  = fig.gca(projection = "3d")
    
    # Plot the surface.
    surf = ax.plot_surface(X, Y, Z, cmap='viridis',linewidth=1) # cmap=cm.coolwarm

    ax.invert_xaxis()
    ax.set_xlabel("Time to maturity ($T-t$)")
    ax.set_ylabel("moneynes ($S_t/K$)")
    ax.set_zlabel("Implied Volatility ($\sigma_t$)")
    ax.set_title(f"Volatility Surface on {year}-{month}-{day}")

    ax.view_init(E,A)
    plt.show()

# plt_vol_surface(0,0)
iplot = interactive(plt_vol_surface, E = (-90, 90, 5), A = (-90, 90, 5))
iplot

interactive(children=(IntSlider(value=0, description='E', max=90, min=-90, step=5), IntSlider(value=0, descrip…

In [ ]:
plt.figure()
plt.plot(pd_underlying_price.Close)
plt.ion()
